In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

Читаем CSV-файлы:

In [2]:
vm = pd.read_csv('vm.csv', index_col=0)
vmType = pd.read_csv('vmType.csv', index_col=0)

Видим, что больше всего виртуальных машин работают с физической машиной номер 16:

In [3]:
vmType.machineId.value_counts()

16    281
11    275
23    242
12    229
21    229
31    201
34    200
3     194
13    194
8     191
1     183
22    182
9     177
27    160
6     157
24    155
30    155
10    123
33    122
29    115
26    115
25    113
2      88
14     86
7      83
5      78
15     75
32     60
19     59
28     43
4      25
0      15
17      6
18      6
20      2
Name: machineId, dtype: int64

Для простоты не будем рассматривать HDD, так как значения для этого ресурса присутствуют не для всех типов виртуальных машин:

In [4]:
vmType.drop(columns=['hdd'], inplace=True)

Оставим только те виртуальные машины, которые размещаются на физической машине с номером 16:

In [5]:
vmType = vmType[vmType['machineId'] == 16].copy()

Столбцы id и machineId тоже не нужны:

In [6]:
vmType.drop(columns=['id', 'machineId'], inplace=True)

Для удобства отсортируем таблицу по vmTypeId:

In [7]:
vmType.sort_values(by=['vmTypeId'], inplace=True, ignore_index=True)

Сделаем индексацию по типу виртуальной машины:

In [8]:
vmType.set_index('vmTypeId', inplace=True)

Из vm уберём столбцы tenantId и priority, они не понадобятся:

In [9]:
vm.drop(columns=['tenantId', 'priority'], inplace=True)

Добавим к vm значения ресурсов и положим получившуюся таблицу в data:

In [10]:
data = pd.merge(vm, vmType, how="left", on="vmTypeId").copy()

Оставим в списке запросов только те виртуальные машины, которые могут быть размещены на физической машине 16:

In [11]:
data = data[data['core'].notna()].copy()

Колонка vmTypeId больше не нужна, удалим её:

In [12]:
data.drop(columns=['vmTypeId'], inplace=True)

Копируем data в data2:

In [13]:
data2 = data.copy()

В data удалим endtime, а starttime переименуем в time. В data2 удалим starttime, а endtime переименуем в time.

In [14]:
data.drop(columns=['endtime'], inplace=True)
data.rename(columns={'starttime' : 'time'}, inplace=True)

data2.drop(columns=['starttime'], inplace=True)
data2.rename(columns={'endtime' : 'time'}, inplace=True)

Добавим в каждую из двух таблиц колонку start_or_end. В data проставим значение start, а в data2 проставим значение end.

In [15]:
data['start_or_end'] = 'start'

data2['start_or_end'] = 'end'

Соединим две таблицы:

In [16]:
data = pd.concat([data, data2], ignore_index=True).copy()

Все строки с NaN (они есть только в столбце time) уберём, так как это просто значит, что виртуальная машина продолжает работать:

In [17]:
data.dropna(inplace=True)

Отсортируем записи по времени, чтобы построчным чтением можно было имитировать динамические запросы:

In [18]:
data.sort_values(by=['time'], inplace=True, ignore_index=True)

Переставим колонки для приведения к требуемому формату:

In [19]:
data = data[['vmId', 'start_or_end', 'time', 'core', 'memory', 'ssd', 'nic']].copy()

Сгенерируем итоговые CSV-файлы:

In [20]:
ROWS = data.shape[0]

In [21]:
import random

In [22]:
random.seed(1011)
for i in range(100):
    start_slice = random.randint(0, ROWS - 10000)
    end_slice = start_slice + 10000 - 1
    data_slice = data.loc[start_slice:end_slice].copy()
    data_slice.to_csv("slices/slice" + str(i + 1) + ".csv", header=False, index=False)